In [1]:
import video_transect, misc
from pathlib import Path
import pandas as pd
import geopandas

In [28]:
# Paths
transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T1'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4')]
# csv_file  = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/GPS_Tracks/skippo_tracks/20230621_skippo_tracks_position_time_only.csv')
csv_file  = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/GPS_Tracks/skippo_tracks/20230621_skippo_tracks_maholmen.csv')



In [29]:
# Parameters
sample_distance = 1.0 # Units defined by CRS, default is UTM with unit meters

In [30]:
def yx_csv_to_geodataframe(csv_file,time_column_name='Time'):
    """ Read position and timestamp from "YX" CSV file (e.g. exported from QGIS)

    # Usage:
    xy_csv_to_geodataframe(csv_file)

    # Required arguments:
    csv_file:   CSV file with track data
                Columns "Y" and "X" and used, in addition to time column
                with label given by keyword argument time_column_name

    # Keyword arguments:
    time_column_name:  String representing column name (header) for time in CSV file

    # Returns
    gdf:        GeoDataFrame with information extracted from CSV file
                Columns are renamed "Lat", "Lng" and "Time"

    """

    # Read data
    data = pd.read_csv(csv_file,usecols=['Y','X',time_column_name])
    data.columns = ['Lat','Lng','Time']                   # Rename columns
    data.Time = pd.to_datetime(data.Time,utc=True)        # Convert time string to datetime format

    # Create GeoDataFrame
    gdf = geopandas.GeoDataFrame(
        data,
        crs = 'EPSG:4326',
        geometry=geopandas.points_from_xy(data.Lng, data.Lat))

    # Return
    return gdf

# Note
Skippo CSV file does not seem to include the first transect. Thus, the first timestamps of Skippo tracks are probably some time after (approx 1 hour?) the timestamps in the videos. 

In [35]:
# Read "Skippo" CSV file
skippo_gdf = yx_csv_to_geodataframe(csv_file,time_column_name='DateTimeS')
skippo_gdf = skippo_gdf.dropna() # Drop points without timestamps (at end of file)
skippo_gdf = skippo_gdf.sort_values('Time')
skippo_gdf

,Lat,Lng,Time,geometry
228,63.409889,7.73651,2023-06-21 11:12:25+00:00,POINT (7.73651 63.40989)
229,63.409882,7.73651,2023-06-21 11:12:27+00:00,POINT (7.73651 63.40988)
230,63.409870,7.73652,2023-06-21 11:12:29+00:00,POINT (7.73652 63.40987)
231,63.409859,7.73653,2023-06-21 11:12:31+00:00,POINT (7.73653 63.40986)
232,63.409851,7.73651,2023-06-21 11:12:33+00:00,POINT (7.73651 63.40985)
...,...,...,...,...
223,63.407139,7.74505,2023-06-21 13:34:09+00:00,POINT (7.74505 63.40714)
224,63.407150,7.74509,2023-06-21 13:34:11+00:00,POINT (7.74509 63.40715)
225,63.407181,7.74515,2023-06-21 13:34:13+00:00,POINT (7.74515 63.40718)
226,63.407242,7.74532,2023-06-21 13:34:15+00:00,POINT (7.74532 63.40724)


In [41]:
# NOTE: Do a "dry run" without extracting images and check that time offsets are correct

for transect_dir in transect_dirs:
    video_dir = transect_dir / 'Video'
    
    image_dir = transect_dir / 'ExtractedImages/Original'
    image_dir.mkdir(parents=True, exist_ok=True)
    print(f'{image_dir} created (if not created before)')

    gpkg_dir = transect_dir / 'ImagesGeopackage'
    gpkg_dir.mkdir(exist_ok=True)
    print(f'{gpkg_dir} created (if not created before)')

    gpkg_file = gpkg_dir / (transect_dir.stem + '_GeotaggedImages.gpkg')
    print(f'GPKG file name for saving results: {gpkg_file}')

    video_files = misc.file_pattern_search(str(video_dir), '*.[Mm][Pp]4')
    print(f'Found video files {[Path(vfile).name for vfile in video_files]}')
    video_data = video_transect.get_video_data(str(video_dir),tz='utc',video_time_offset=pd.Timedelta(hours=-2))

    #display(video_data)
    
    gdf = video_transect.prepare_gdf_with_video_data(skippo_gdf,video_data)
    display(gdf)

    #gdf = video_transect.extract_images_from_video(gdf,str(image_dir))
    #gdf.to_file(gpkg_file, driver="GPKG")
    

/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T1/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T1/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T1/ImagesGeopackage/T1_GeotaggedImages.gpkg
Found video files ['GH010770.MP4', 'GH020770.MP4', 'GH030770.MP4', 'GH040770.MP4', 'GH050770.MP4', 'GH060770.MP4']


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry


/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ImagesGeopackage/T2_GeotaggedImages.gpkg
Found video files ['GH010771.MP4', 'GH020771.MP4', 'GH030771.MP4', 'GH040771.MP4', 'GH050771.MP4']


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry
228,63.409889,7.73651,2023-06-21 11:12:25+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:29,29.000,POINT (7.73651 63.40989)
229,63.409882,7.73651,2023-06-21 11:12:27+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:31,31.000,POINT (7.73651 63.40988)
230,63.409870,7.73652,2023-06-21 11:12:29+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:33,33.000,POINT (7.73652 63.40987)
231,63.409859,7.73653,2023-06-21 11:12:31+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:35,35.000,POINT (7.73653 63.40986)
232,63.409851,7.73651,2023-06-21 11:12:33+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:37,37.000,POINT (7.73651 63.40985)
...,...,...,...,...,...,...,...
1487,63.410801,7.72811,2023-06-21 11:54:23+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:42:27,420.876,POINT (7.72811 63.41080)
1488,63.410789,7.72810,2023-06-21 11:54:25+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:42:29,422.876,POINT (7.72810 63.41079)
1489,63.410782,7.72808,2023-06-21 11:54:27+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:42:31,424.876,POINT (7.72808 63.41078)
1490,63.410770,7.72806,2023-06-21 11:54:29+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:42:33,426.876,POINT (7.72806 63.41077)


/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ImagesGeopackage/T3_GeotaggedImages.gpkg
Found video files ['GH010772.MP4', 'GH020772.MP4', 'GH030772.MP4']


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry
1604,63.410042,7.72543,2023-06-21 11:58:17+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:01,1.000,POINT (7.72543 63.41004)
1605,63.410042,7.72540,2023-06-21 11:58:19+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:03,3.000,POINT (7.72540 63.41004)
1606,63.410030,7.72538,2023-06-21 11:58:21+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:05,5.000,POINT (7.72538 63.41003)
1607,63.410019,7.72536,2023-06-21 11:58:23+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:07,7.000,POINT (7.72536 63.41002)
1608,63.410011,7.72534,2023-06-21 11:58:25+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:00:09,9.000,POINT (7.72534 63.41001)
...,...,...,...,...,...,...,...
2056,63.409199,7.73037,2023-06-21 12:13:21+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:15:05,373.469,POINT (7.73037 63.40920)
2057,63.409199,7.73042,2023-06-21 12:13:23+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:15:07,375.469,POINT (7.73042 63.40920)
2058,63.409191,7.73048,2023-06-21 12:13:25+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:15:09,377.469,POINT (7.73048 63.40919)
2059,63.409180,7.73052,2023-06-21 12:13:27+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:15:11,379.469,POINT (7.73052 63.40918)


/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ImagesGeopackage/T4_GeotaggedImages.gpkg
Found video files ['GH010773.MP4', 'GH020773.MP4']


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry
0,63.407001,7.74536,2023-06-21 13:26:43+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:03:39,219.000,POINT (7.74536 63.40700)
1,63.406990,7.74534,2023-06-21 13:26:45+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:03:41,221.000,POINT (7.74534 63.40699)
2,63.406990,7.74531,2023-06-21 13:26:47+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:03:43,223.000,POINT (7.74531 63.40699)
3,63.406979,7.74529,2023-06-21 13:26:49+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:03:45,225.000,POINT (7.74529 63.40698)
4,63.406979,7.74525,2023-06-21 13:26:51+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:03:47,227.000,POINT (7.74525 63.40698)
...,...,...,...,...,...,...,...
223,63.407139,7.74505,2023-06-21 13:34:09+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:11:05,133.469,POINT (7.74505 63.40714)
224,63.407150,7.74509,2023-06-21 13:34:11+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:11:07,135.469,POINT (7.74509 63.40715)
225,63.407181,7.74515,2023-06-21 13:34:13+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:11:09,137.469,POINT (7.74515 63.40718)
226,63.407242,7.74532,2023-06-21 13:34:15+00:00,/media/mha114/Massimal2/Massimal/Smola_Maholme...,0 days 00:11:11,139.469,POINT (7.74532 63.40724)
